<a href="https://colab.research.google.com/github/VL98/VLehner-Bachelor-Thesis/blob/main/Esscher_Calibration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Esscher Pricing Calibration

R Script by Vinzenz Lehner (May 2021)

### Step 1: 
Installing, loading required libraries

In [ ]:
install.packages(c("nloptr", "rootSolve", "pracma", "NlcOptim","parallel", "ragtop", "timeSeries"))

library(ragtop)
library(pracma)
library(nloptr)
library(NlcOptim)
library(rootSolve)
library(stats)
library(parallel)
library(timeSeries)

seed = set.seed(1004)



### Step 2:
Defining global variables


In [ ]:
seed = set.seed(1004)
option = read.csv("option15clean.csv")
v = unique(option$date)
n = 10^5
x_2 = read.csv("firstpar.csv")
 x_2 = as.numeric(x_2$x[2:12])
gnoise = rnorm(n*151) 
sa = sample(1:(n*151))

### Step 3:
Defining functions


In [ ]:
scaler = function(x){
  x/(10^ceil(log10(abs(x_2))))
}

rescaler = function(x){
  x*(10^ceil(log10(abs(x_2))))
}



hin3 = function(x) {
  x = rescaler(x)
  h = numeric(1)
  h[1] = x[2]*(x[4]+ x[11] + 2)^2  + x[3] - 1 + 1e-15
  return(h)
}

heq3 = function(x) {
  x = rescaler(x)
  h = numeric(2)
  h[1] = x[10]*x[6] + (1-x[10])*x[7]
  h[2] =  x[10]*(x[6]^2)  + (1-x[10])*(x[7]^2)  + x[10]*(x[8]^2) + (1-x[10])*(x[9]^2) - 1
  return(h)
}


scaler(x_2)

In [ ]:
fN3 = function(x){
  x = rescaler(x)
  n = 100000                ######### parameters of the model
  set.seed(400)
  t_size = option$date == date
  date = date
  om = x[1]
  a = x[2]
  b = x[3]
  g = x[4]
  l = x[11]
  h_0 = x[5]
  mu_1 = x[6]
  mu_2 = x[7]
  sd_1 = x[8]
  sd_2 = x[9]
  p = x[10]
  E = p*mu_1  +  (1-p)*mu_2    ######## mean
  
  H = p*(mu_1^2)  + (1-p)*(mu_2^2)  + p*(sd_1^2) + (1-p)*(sd_2^2) - E^2   #### variance
  
  ##M_1 = sqrt(1/H)*mu_1 - sqrt(1/H)*E
  M_1 = mu_1                                  ###### real mu_1: there is a second option to incoroporate the constraint
  ## M_2 = sqrt(1/H)*mu_2 - sqrt(1/H)*E
 M_2 = mu_2
   
  ## S_1 = sd_1/sqrt(H)
 S_1 = sd_1                              #### same with the variance
  ## S_2 = sd_2/sqrt(H)
  S_2 = sd_2    
  rate = function(z){option[ option$date == date & option$TTM == z,  ]$interest[1]/(100*252)}
  d = unique(option$TTM[t_size])
  r0 = median(sapply(d, rate))
  
  G = function(u , v, r = r0, mean = T){
    
    m = (r + v*l) * mean
    
    y = p*exp((m + sqrt(v)*M_1)*u  + (u^2)*(S_1^2)*v/2) +
      (1-p)*exp((m + sqrt(v)*M_2)*u  + (u^2)*(S_2^2)*v/2) 
    
    return(y)
  }
  
  
  
  f = function(u, v) {
  ( log(G(u+1, v)/G(u, v)) - r0)
    
  }
  
  
  
  h =  matrix(data = 0, nrow = n, ncol = max(d))
  
  h[ ,1] = h_0
  
  
  psize = 1:round(p*n*151)
  noise = c((M_1 + S_1*gnoise[psize]),( M_2 + S_2*gnoise[-psize]))[sa]
  e = matrix(data = noise[1:(max(d)*n)], nrow = 100000, ncol = max(d))
  
  
  for(t in 2:(max(d))){   ### carefull with the e
    
    h[ , t] = om + a*(e[ ,t-1] - g*sqrt(h[ ,t-1]))^2 + b*h[ ,t-1]
  }
  m = 350
  ## if(is.nan(h[1, 1])){ return(10^5)} #### checking if the evaluation produces a finite output
  
  vec = runif(n = m, min = min(h), max = max(h))
  zi = function(x){ f(x, vec)}
  vec2 =   multiroot(zi, start = rep(-1.695, m), maxiter = 1000, rtol = 1e-5, atol = 1e-7 , ctol = 1e-7, verbose = FALSE)$root
  thf = approxfun(vec, vec2, rule = c(2,2))
  s1 = sqrt(h)*e[ , 1:max(d)] + l*h
  th =  matrix(thf(h), ncol = max(d), nrow = n)
  z0 = th*s1
  
  
  ##### experiments for the creation of S and Z
  ##T_m = d[1]   ## for testing
  zfun = function(T_m){
    r = rate(T_m)
    ku = G(th[ , 1:T_m], h[ , 1:T_m], r)
    S_0 = option[ option$date == date & option$TTM == T_m,  ]$SPOT[1]
    Z = matrix(0, ncol = T_m, nrow = n)
    S = matrix(0, ncol = T_m, nrow = n)
    Z[ ,1] = exp(r*th[ ,1] + z0[ ,1])/ku[ ,1]
    Z[ ,1] = Z[ ,1]/mean(Z[ ,1])
    S[ , 1] = S_0*exp(r + s1[ ,1])
    S[ , 1] = S[ , 1] *S_0*exp(r)/mean(S[ , 1]*Z[ ,1])
    for( t in 2:T_m){
      Z[ ,t] = Z[ ,t-1]*exp(r*th[ ,t] + z0[ ,t])/ku[ ,t]
      Z[ ,t] =  Z[ ,t]/mean( Z[ ,t])
      S[ , t] = S[ , t-1]*exp(r + s1[ ,t])
      S[ , t] = S[ , t]*S_0*exp(r*t)/mean(S[ , t]*Z[ ,t])
    }
    list(S[ ,T_m], Z[ ,T_m])
    
  }
  
  ##  mean(Z[, T_m]*S[ ,T_m ])/exp(r*T_m) ### for testing if results make any sense
  ## mean(Z[, T_m])
  ##### this function creates the RND for all maturities
  
  ZL = mclapply(d, zfun, mc.cores = 4)
  
  ## y = option[1, c("strike_price", "SPOT", "TTM") ]
  A = function(y){ #######!!!!!!!! be careful with the indexes in the data set
    y = as.numeric(y)
    T_m = y[3]
    S_0 = y[2] ### or futures price
    r = rate(T_m)
    i = sum((d == T_m)* (1:length(d)))
    Z = ZL[[i]][[2]] ##
    S =  ZL[[i]][[1]]
    K = y[1]
    
    C = mean((S > K)*(S - K)*exp(-r*T_m)*Z)
    IV = try( implied_volatilities(C, 1, S_0, K, r*252, T_m/252))
    ifelse(class(IV) == "try-error"| is.na(IV), 4, IV)
    
  }
  
  
  CP = apply(option[t_size, c("strike_price", "SPOT", "TTM") ],1, A)
  ##mean((CP - option$price[t_size])^2)
  mean((CP - option[t_size, "own_impvol"])^2)
}


Defining functions (Part III)

In [ ]:
FN3_C_Final = function(x){
  a = try(fN3(x))
  ifelse(class(a)== "try-error"| is.na(a), 16, a)
}


### Step 4:

Performing calibration

In [ ]:
xN = rep(0, 11)
start = Sys.time()
for(i in 12:18) {
  date = v[i]
boundslow = c(1e-12, 0, 0, -1500, 1e-12, -15,  -15, 1e-12, 1e-12, 0, -20 ) 
boundsup  = c(0.6, 0.6, 1, 1500, 1, 15, 15, 10, 20, 1, 20)


Model = fmincon(scaler(x_2), FN3_C_Final, lb = scaler(boundslow), ub = scaler(boundsup), hin =  hin3, heq = heq3, tol = 1e-06, 
                        maxfeval = 10000, maxiter = 5000 )
Model$par = rescaler(Model$par)


xN =  rbind(xN ,Model$par)
}

dur = Sys.time() - start

In [ ]:
#c = c(date, Model$par)
date = v[18]
fN3R(xN[ 8, ])
#x_2 = Model$par
c = cbind(v[12:18], xN[2:8, ])
write.csv(c, "par12-18.csv")

In [ ]:
date = v[11]
#fN3R(xN[2, ])
XN = matrix(xN[12:33],  nrow = 2, ncol = 11)
##x3 = cbind(v[3:8], x[2:7, ])
##write.csv(x3, "par3:8.csv")
XN[1 , ] = xN[12:22]
XN[2 , ] = xN[23:33]
write.csv(XN, "par10-11.csv")

In [ ]:
x = rep(0,11)

#Model[i , ] = c(date, Model$par,fN3R(Model$par))

x = rbind(x, rep(1,11))
x

In [ ]:
fN3R = function(x){

 
    n = 100000                ######### parameters of the model
    set.seed(400)
    t_size = option$date == date
    date = date
    om = x[1]
    a = x[2]
    b = x[3]
    g = x[4]
    l = x[11]
    h_0 = x[5]
    mu_1 = x[6]
    mu_2 = x[7]
    sd_1 = x[8]
    sd_2 = x[9]
    p = x[10]
    E = p*mu_1  +  (1-p)*mu_2    ######## mean
    
    H = p*(mu_1^2)  + (1-p)*(mu_2^2)  + p*(sd_1^2) + (1-p)*(sd_2^2) - E^2   #### variance
    
   ## M_1 = sqrt(1/H)*mu_1 - sqrt(1/H)*E
  M_1 = mu_1                                  ###### real mu_1: there is a second option to incoroporate the constraint
    ##M_2 = sqrt(1/H)*mu_2 - sqrt(1/H)*E
      M_2 = mu_2
   ## S_1 = sd_1/sqrt(H)
    S_1 = sd_1                              #### same with the variance
    ##S_2 = sd_2/sqrt(H)
    S_2 = sd_2    
    rate = function(z){option[ option$date == date & option$TTM == z,  ]$interest[1]/(100*252)}
    d = unique(option$TTM[t_size])
    r0 = median(sapply(d, rate))
    
    G = function(u , v, r = r0, mean = T){
      
      m = (r + v*l) * mean
      
      y = p*exp((m + sqrt(v)*M_1)*u  + (u^2)*(S_1^2)*v/2) +
        (1-p)*exp((m + sqrt(v)*M_2)*u  + (u^2)*(S_2^2)*v/2) 
      
      return(y)
    }
    
    
    
    f = function(u, v) {
      ( log(G(u+1, v)/G(u, v)) - r0)
      
    }
    
    
    
    h =  matrix(data = 0, nrow = n, ncol = max(d))
    
    h[ ,1] = h_0
    
    
    psize = 1:round(p*n*151)
    noise = c((M_1 + S_1*gnoise[psize]),( M_2 + S_2*gnoise[-psize]))[sa]
    e = matrix(data = noise[1:(max(d)*n)], nrow = 100000, ncol = max(d))
    
    
    for(t in 2:(max(d))){   ### carefull with the e
      
      h[ , t] = om + a*(e[ ,t-1] - g*sqrt(h[ ,t-1]))^2 + b*h[ ,t-1]
    }
    m = 350
    ## if(is.nan(h[1, 1])){ return(10^5)} #### checking if the evaluation produces a finite output
    
    vec = runif(n = m, min = min(h), max = max(h))
    zi = function(x){ f(x, vec)}
    vec2 =  multiroot(zi, start = rep(-1.695, m), maxiter = 1000, rtol = 1e-5, atol = 1e-7 , ctol = 1e-7)$root
    thf = approxfun(vec, vec2, rule = c(2,2))
    s1 = sqrt(h)*e[ , 1:max(d)] + l*h
    th =  matrix(thf(h), ncol = max(d), nrow = n)
    z0 = th*s1
    
    
    ##### experiments for the creation of S and Z
    ##T_m = d[1]   ## for testing
    zfun = function(T_m){
      r = rate(T_m)
      ku = G(th[ , 1:T_m], h[ , 1:T_m], r)
      S_0 = option[ option$date == date & option$TTM == T_m,  ]$SPOT[1]
      Z = matrix(0, ncol = T_m, nrow = n)
      S = matrix(0, ncol = T_m, nrow = n)
      Z[ ,1] = exp(r*th[ ,1] + z0[ ,1])/ku[ ,1]
      Z[ ,1] = Z[ ,1]/mean(Z[ ,1])
      S[ , 1] = S_0*exp(r + s1[ ,1])
      S[ , 1] = S[ , 1] *S_0*exp(r)/mean(S[ , 1]*Z[ ,1])
      for( t in 2:T_m){
        Z[ ,t] = Z[ ,t-1]*exp(r*th[ ,t] + z0[ ,t])/ku[ ,t]
        Z[ ,t] =  Z[ ,t]/mean( Z[ ,t])
        S[ , t] = S[ , t-1]*exp(r + s1[ ,t])
        S[ , t] = S[ , t]*S_0*exp(r*t)/mean(S[ , t]*Z[ ,t])
      }
      list(S, Z)
      
    }
    
    ##  mean(Z[, T_m]*S[ ,T_m ])/exp(r*T_m) ### for testing if results make any sense
    ## mean(Z[, T_m])
    ##### this function creates the RND for all maturities
    
    ZL = mclapply(d, zfun, mc.cores = 4) ### be careful with the number of cores
    
    ## y = option[1, c("strike_price", "SPOT", "TTM") ]
    A = function(y){ #######!!!!!!!! be careful with the indexes in the data set
      y = as.numeric(y)
      T_m = y[3]
      S_0 = y[2] ### or futures price
      r = rate(T_m)
      i = sum((d == T_m)* (1:length(d)))
      Z = ZL[[i]][[2]] ##
      S =  ZL[[i]][[1]]
      K = y[1]
      
      C = mean((S[ ,T_m] > K)*(S[ ,T_m] - K)*exp(-r*T_m)*Z[ ,T_m])
      
      ifelse(class(C) == "try-error"| is.na(C), 6, C)
      
    }
    
    
    CP = apply(option[t_size, c("strike_price", "SPOT", "TTM") ],1, A)
    
    mean((CP - option[t_size, "price"])^2)
   # CP
  }
  

In [ ]:
#a = (fN3R(Model$par) - option[option$date == date, "price" ]) == max(fN3R(Model$par) - option[option$date == date, "price" ])
##option[a & option$date == date , ]
##fN3R(Model$par)[a]

fN3R(Model$par) 
option#[option$date == date  ,  ]

In [ ]:

write.csv(c(date, Model3$par,fN3R(Model3$par)), "thirdpar.csv")

x_2 = Model$par
##saveRDS(Model, "Model1.rds")
##Model$date = date
##write.csv(Model$par, "x0")

In [ ]:
install.packages("gargle")

In [ ]:
library("gargle")

In [ ]:
library("googledrive")

In [ ]:
drive_auth(
  email = gargle::gargle_oauth_email(),
  path = NULL,
  scopes = "https://www.googleapis.com/auth/drive",
  cache = gargle::gargle_oauth_cache(),
  use_oob = gargle::gargle_oob_default(),
  token = NULL
)

In [ ]:
save(Model, file="Model.RData")

In [ ]:
fN3 = function(x){
  x = rescaler(x)
  n = 100000                ######### parameters of the model
  set.seed(400)
  t_size = option$date == date
  date = date
  om = x[1]
  a = x[2]
  b = x[3]
  g = x[4]
  l = x[11]
  h_0 = x[5]
  mu_1 = x[6]
  mu_2 = x[7]
  sd_1 = x[8]
  sd_2 = x[9]
  p = x[10]
  E = p*mu_1  +  (1-p)*mu_2    ######## mean
  
  H = p*(mu_1^2)  + (1-p)*(mu_2^2)  + p*(sd_1^2) + (1-p)*(sd_2^2) - E^2   #### variance
  
  M_1 = sqrt(1/H)*mu_1 - sqrt(1/H)*E
   ##M_1 = mu_1                                  ###### real mu_1: there is a second option to incoroporate the constraint
 M_2 = sqrt(1/H)*mu_2 - sqrt(1/H)*E
  ##M_2 = mu_2
   
   S_1 = sd_1/sqrt(H)
  ##S_1 = sd_1                              #### same with the variance
   S_2 = sd_2/sqrt(H)
 ## S_2 = sd_2    
  rate = function(z){option[ option$date == date & option$TTM == z,  ]$interest[1]/(100*252)}
  d = unique(option$TTM[t_size])
  r0 = median(sapply(d, rate))
  
  G = function(u , v, r = r0, mean = T){
    
    m = (r + v*l) * mean
    
    y = p*exp((m + sqrt(v)*M_1)*u  + (u^2)*(S_1^2)*v/2) +
      (1-p)*exp((m + sqrt(v)*M_2)*u  + (u^2)*(S_2^2)*v/2) 
    
    return(y)
  }
  
  
  
  f = function(u, v) {
  ( log(G(u+1, v)/G(u, v)) - r0)
    
  }
  
  
  
  h =  matrix(data = 0, nrow = n, ncol = max(d))
  
  h[ ,1] = h_0
  
  
  psize = 1:round(p*n*254)
  noise = c((M_1 + S_1*gnoise[psize]),( M_2 + S_2*gnoise[-psize]))[sa]
  e = matrix(data = noise[1:(max(d)*n)], nrow = 100000, ncol = max(d))
  
  
  for(t in 2:(max(d))){   ### carefull with the e
    
    h[ , t] = om + a*(e[ ,t-1] - g*sqrt(h[ ,t-1]))^2 + b*h[ ,t-1]
  }
  m = 350
  ## if(is.nan(h[1, 1])){ return(10^5)} #### checking if the evaluation produces a finite output
  
  vec = runif(n = m, min = min(h), max = max(h))
  zi = function(x){ f(x, vec)}
  vec2 =   multiroot(zi, start = rep(-1.695, m), maxiter = 1000, rtol = 1e-5, atol = 1e-7 , ctol = 1e-7, verbose = FALSE)$root
  thf = approxfun(vec, vec2, rule = c(2,2))
  s1 = sqrt(h)*e[ , 1:max(d)] + l*h
  th =  matrix(thf(h), ncol = max(d), nrow = n)
  z0 = th*s1
  
  
  ##### experiments for the creation of S and Z
  ##T_m = d[1]   ## for testing
  zfun = function(T_m){
    r = rate(T_m)
    ku = G(th[ , 1:T_m], h[ , 1:T_m], r)
    S_0 = option[ option$date == date & option$TTM == T_m,  ]$SPOT[1]
    Z = matrix(0, ncol = T_m, nrow = n)
    S = matrix(0, ncol = T_m, nrow = n)
    Z[ ,1] = exp(r*th[ ,1] + z0[ ,1])/ku[ ,1]
    Z[ ,1] = Z[ ,1]/mean(Z[ ,1])
    S[ , 1] = S_0*exp(r + s1[ ,1])
    S[ , 1] = S[ , 1] *S_0*exp(r)/mean(S[ , 1]*Z[ ,1])
    for( t in 2:T_m){
      Z[ ,t] = Z[ ,t-1]*exp(r*th[ ,t] + z0[ ,t])/ku[ ,t]
      Z[ ,t] =  Z[ ,t]/mean( Z[ ,t])
      S[ , t] = S[ , t-1]*exp(r + s1[ ,t])
      S[ , t] = S[ , t]*S_0*exp(r*t)/mean(S[ , t]*Z[ ,t])
    }
    list(S, Z)
    
  }
  
  ##  mean(Z[, T_m]*S[ ,T_m ])/exp(r*T_m) ### for testing if results make any sense
  ## mean(Z[, T_m])
  ##### this function creates the RND for all maturities
  
  ZL = mclapply(d, zfun, mc.cores = 4)
  
  ## y = option[1, c("strike_price", "SPOT", "TTM") ]
  A = function(y){ #######!!!!!!!! be careful with the indexes in the data set
    y = as.numeric(y)
    T_m = y[3]
    S_0 = y[2] ### or futures price
    r = rate(T_m)
    i = sum((d == T_m)* (1:length(d)))
    Z = ZL[[i]][[2]] ##
    S =  ZL[[i]][[1]]
    K = y[1]
    
    C = mean((S[ ,T_m] > K)*(S[ ,T_m] - K)*exp(-r*T_m))
    IV = try( implied_volatilities(C, 1, S_0, K, r*252, T_m/252))
    ifelse(class(IV) == "try-error"| is.na(IV), 6, IV)
    
  }
  
  
  CP = apply(option[t_size, c("strike_price", "SPOT", "TTM") ],1, A)
  ##mean((CP - option$price[t_size])^2)
  mean((CP - option[t_size, "own_impvol"])^2)
}


In [ ]:
funcons2 = function(x) {
  x = rescaler(x)
  h = numeric(9)
  h[1] = -x[2]*(x[4]+ x[11]+ 2)^2  + -x[3] + 1 - 1e-20
  h[2] = x[2]
  h[3] = x[3] 
  h[4] = x[1]  - 1e-20
  h[5] = x[5] - 1e-20
  h[6] = x[10]
  h[7] = 1 - x[10]
  h[8] = x[8] - 1e-20
  h[9] = x[9] - 1e-20
  return(h)
}


calcfun = function(x){
  mu_1 = x[1]
mu_2 = x[2]
sd_1 = x[3]
sd_2 = x[4]
E = p*mu_1  +  (1-p)*mu_2   
H = p*(mu_1^2)  + (1-p)*(mu_2^2)  + p*(sd_1^2) + (1-p)*(sd_2^2) - E^2

M_1 = sqrt(1/H)*mu_1 - sqrt(1/H)*E
###### real mu_1: there is a second option to incoroporate the constraint
M_2 = sqrt(1/H)*mu_2 - sqrt(1/H)*E

S_1 = sd_1/sqrt(H)
#### same with the variance
S_2 = sd_2/sqrt(H)

c(M_1, M_2, S_1, S_2)

}
durs =rep(0, length(v))
start = Sys.time()
Model4 = cobyla(x0 = scaler(x_2), fn  = FN3_C_Final,  hin = funcons2, nl.info =  T,
                control = list(xtol_rel = 1e-06, maxeval = 50))
dur = Sys.time() - start
Model4$par = rescaler(Model5$par)
Model4$par[6:9] = calcfun(Model5$par[6:9])

### Step 6:

Defining function for Performance measurement 

In [ ]:
fN3R2 = function(x){
 
    n = 100000                ######### parameters of the model
    set.seed(400)
    t_size = option$date == date
    date = date
    om = x[1]
    a = x[2]
    b = x[3]
    g = x[4]
    l = x[11]
    h_0 = x[5]
    mu_1 = x[6]
    mu_2 = x[7]
    sd_1 = x[8]
    sd_2 = x[9]
    p = x[10]
    E = p*mu_1  +  (1-p)*mu_2    ######## mean
    
    H = p*(mu_1^2)  + (1-p)*(mu_2^2)  + p*(sd_1^2) + (1-p)*(sd_2^2) - E^2   #### variance
    
   ## M_1 = sqrt(1/H)*mu_1 - sqrt(1/H)*E
  M_1 = mu_1                                  ###### real mu_1: there is a second option to incoroporate the constraint
    ##M_2 = sqrt(1/H)*mu_2 - sqrt(1/H)*E
      M_2 = mu_2
   ## S_1 = sd_1/sqrt(H)
    S_1 = sd_1                              #### same with the variance
    ##S_2 = sd_2/sqrt(H)
    S_2 = sd_2    
    rate = function(z){option[ option$date == date & option$TTM == z,  ]$interest[1]/(100*252)}
    d = unique(option$TTM[t_size])
    r0 = median(sapply(d, rate))
    
    G = function(u , v, r = r0, mean = T){
      
      m = (r + v*l) * mean
      
      y = p*exp((m + sqrt(v)*M_1)*u  + (u^2)*(S_1^2)*v/2) +
        (1-p)*exp((m + sqrt(v)*M_2)*u  + (u^2)*(S_2^2)*v/2) 
      
      return(y)
    }
    
    
    
    f = function(u, v) {
      ( log(G(u+1, v)/G(u, v)) - r0)
      
    }
    
    
    
    h =  matrix(data = 0, nrow = n, ncol = max(d))
    
    h[ ,1] = h_0
    
    
    psize = 1:round(p*n*254)
    noise = c((M_1 + S_1*gnoise[psize]),( M_2 + S_2*gnoise[-psize]))[sa]
    e = matrix(data = noise[1:(max(d)*n)], nrow = 100000, ncol = max(d))
    
    
    for(t in 2:(max(d))){   ### carefull with the e
      
      h[ , t] = om + a*(e[ ,t-1] - g*sqrt(h[ ,t-1]))^2 + b*h[ ,t-1]
    }
    m = 350
    ## if(is.nan(h[1, 1])){ return(10^5)} #### checking if the evaluation produces a finite output
    
    vec = runif(n = m, min = min(h), max = max(h))
    zi = function(x){ f(x, vec)}
    vec2 =  multiroot(zi, start = rep(-1.695, m), maxiter = 1000, rtol = 1e-5, atol = 1e-7 , ctol = 1e-7)$root
    thf = approxfun(vec, vec2, rule = c(2,2))
    s1 = sqrt(h)*e[ , 1:max(d)] + l*h
    th =  matrix(thf(h), ncol = max(d), nrow = n)
    z0 = th*s1
    
   S = exp(rowCumsums(s1)) 

  
  

zfun = function(T_m){
  r = rate(T_m)
  s = th[ ,1:T_m]*r + z0[ , 1:T_m] - log(G(th[ , 1:T_m], h[ , 1:T_m], r)) 
  exp(rowCumsums(s))[ , T_m]
}
##### this function creates the RND for all maturities
ZL = mclapply(d, zfun, mc.cores = 6)
  A = function(y){ #######!!!!!!!! be careful with the indexes in the data set
    y = as.numeric(y)
    T_m = y[3]
    S_0 = y[2] ### or futures price
    r = rate(T_m)
    i = sum((d == T_m)* (1:length(d)))
    Z = ZL[[i]] ##Z[ ,T_m]
    R = S[ , T_m] * S_0*exp(r*T_m)
    Z = Z/mean(Z)
    s_m = mean(R*Z)
    K = y[1]
    EMS = S_0*exp(r*T_m)/s_m
    
    B = mean((R*EMS > K)*(R*EMS - K)*exp(-r*T_m)*Z)
    
    ifelse(is.nan(B), 100000,B)
    
  }
    CP = apply(option[t_size, c("strike_price", "SPOT", "TTM") ],1, A)
    ##mean((CP - option$price[t_size])^2)
   ## mean((CP - option[t_size, "price"])^2)
   e
  }

  
      fN3R2(x_2)
     ###fN3R(x_2)